In [17]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
import os

### Load your data

In [18]:
file_path = Path("sleep.epub")
loader = UnstructuredFileLoader("sleep.epub")
data = loader.load() 

### Chunk your data up into smaller documents

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(data)

In [20]:
print (f'Now you have {len(texts)} documents')

Now you have 1722 documents


### Create embeddings of your documents to get ready for semantic search

In [21]:
embeddings = HuggingFaceEmbeddings(model_name="msmarco-roberta-base-v2")

In [22]:
db = Chroma.from_documents(texts, embeddings)

### Query those docs to get your answer back

In [23]:
base_path = os.environ.get('OPENAI_API_BASE', 'http://localhost:8000/v1')

In [24]:
api_key = "your_api_key"
api_base = "http://localhost:8000/v1"
llm = OpenAI(openai_api_base=api_base, openai_api_key=api_key)
chain = load_qa_chain(llm, chain_type="stuff")

In [25]:
query = "My son is three months old. What would be a good sleep schedule for him? Show me an example."
docs = db.similarity_search(query, include_metadata=True)
results_1 = chain.run(input_documents=docs, question=query)

In [26]:
print(results_1) 

 A good sleep schedule for your 3-month-old may include the following routine:
11 pm – Bedtime
12 am – Awake (feeding, diaper change)
1 am – Back to bed
3 am – Awake (feeding, diaper change)
4 am – Back to bed
7 am – Wake up


In [27]:
docs

[Document(page_content='HELPFUL HINT\n\nSome children appear to hate their bedroom in the afternoon and scream as you approach it. One mother solved this by doing all the pre-nap soothing in the living room and then quickly went into her child’s room.\n\nQ: How long should my child nap?\n\nA: Does your child appear well rested? You be the judge. All of us have good days and bad days, but if you notice a progression toward more fussiness, brattiness, or tantrums, your child may need longer naps.\n\nMonths 15 to 21\n\nNAP #1 (MIDMORNING NAP) BEGINS TO DISAPPEAR', metadata={'source': 'sleep.epub'}),
 Document(page_content='This lack of rigid scheduling is appropriate for children a few months old, who are biologically immature. However, as the child gets older, extreme inconsistency will produce unhealthy sleep habits. Be flexible, but also become sensitive to your child’s need to sleep. Remember, it’s your responsibility as a parent to provide structure for your child: she cannot do it o